In [3]:
import tensorflow_datasets as tfds

c:\Users\teera\anaconda3\envs\env_tfdv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from tensorflow import keras

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout( 0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 14s 6ms/step - loss: 0.2992 - accuracy: 0.9136
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1424 - accuracy: 0.9579
Epoch 3/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.1063 - accuracy: 0.9665
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0859 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0746 - accuracy: 0.9761
313/313 - 2s - loss: 0.0766 - accuracy: 0.9756 - 2s/epoch - 8ms/step


[0.07656548172235489, 0.975600004196167]

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


model = MyModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='test_accuracy')


@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

Epoch 1, Loss: 0.13926507532596588, Accuracy: 95.81999969482422, Test Loss: 0.05634070187807083, Test Accuracy: 98.0999984741211
Epoch 2, Loss: 0.0443207286298275, Accuracy: 98.61500549316406, Test Loss: 0.05143485963344574, Test Accuracy: 98.48999786376953
Epoch 3, Loss: 0.02366667054593563, Accuracy: 99.24500274658203, Test Loss: 0.050035230815410614, Test Accuracy: 98.29999542236328


KeyboardInterrupt: 

In [2]:
!pip install colab

     ---------------------------------------- 0.0/567.7 kB ? eta -:--:--
      --------------------------------------- 10.2/567.7 kB ? eta -:--:--
      --------------------------------------- 10.2/567.7 kB ? eta -:--:--
     -- ---------------------------------- 41.0/567.7 kB 219.4 kB/s eta 0:00:03
     --------------- ---------------------- 225.3/567.7 kB 1.1 MB/s eta 0:00:01
     -------------------------------------  563.2/567.7 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 567.7/567.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      error in colab setup command: 'install_requires' must be a string or list of strings containing valid project/version requirement specifiers; Expected end or semicolon (after version specifier)
          pytz>=2011n
              ~~~~~~^
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import colab